## Test lecture data sur S3

In [21]:
import boto3
import pandas as pd
from io import BytesIO

In [22]:
# Nom du bucket et chemin du fichier
bucket_name = "david-mlops-bucket"
fichier_s3  = "s3://david-mlops-bucket/conso grdf/Donnees_informatives_02269609199626_2025-10-13_2025-10-20_68f65c45bfe56.xlsx"

import awswrangler as wr

#s3_path = 's3://votre-bucket/chemin/vers/votre/fichier.xlsx'
#df = wr.s3.read_excel(path=fichier_s3 , sheet_name=0, skiprows=8,usecols="B:H") # sheet_name=0 pour la première feuille

In [23]:
import boto3

def lister_fichiers_excel_s3():
    """
    Récupère la liste des fichiers Excel (.xlsx, .xls) dans un bucket S3.

    :return: Une liste des clés (noms de chemins) des fichiers Excel.
    """
    import os

    bucket_name   = os.environ.get("NOM_DU_BUCKET")
    prefix        = os.environ.get("PREFIXE_DOSSIER")  

    s3 = boto3.client(
        's3',
        aws_access_key_id     = os.environ.get("aws_access_key_id"),
        aws_secret_access_key = os.environ.get("aws_secret_access_key")
    )

    
    excel_files = []
    
    # Pour gérer un grand nombre de fichiers (pagination)
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                # Filtrer par extension
                if key.lower().endswith('.xlsx') or key.lower().endswith('.xls'):                    
                    excel_files.append(f"s3://{bucket_name}/{key}")
                    
    return excel_files


## Injection données météo